In [ ]:
from Bio.PDB import PDBParser
import pyKVFinder
parser = PDBParser()
import os


import nglview as nv


In [ ]:
import mdtraj as md
pdb='../segment_5_input_af_0_sys.pdb'
xtc='../segment_5_input_af_rebuilt.xtc'
traj = md.load(xtc, top=pdb)
traj.save('ensemble.xtc')

for i in range(0,len(traj),2):
    traj[i].save('data/'+str(i)+'_a.pdb')
    

In [ ]:
from os import listdir
pdbs=os.listdir("./data/")
for i in range(0,len(pdbs)):
    toml_f='results_'+str(pdbs[i].replace('.pdb', ''))+'.toml'
    cavity_f='cavity_'+str(pdbs[i])
    print(toml_f,cavity_f)
    try:
        results = pyKVFinder.run_workflow('data/'+pdbs[i], include_depth=True, include_hydropathy=True, hydrophobicity_scale='EisenbergWeiss')
        results.avg_depth
        results.export_all(fn='results/'+str(toml_f), output='cavities/'+str(cavity_f), include_frequencies_pdf=False)
    except Exception as e:
        pass

In [ ]:
#Rmsf & secondary filter
import numpy as np
x=np.loadtxt("rmsf/rmsf.xvg")[:, 0]
y=np.loadtxt("rmsf/rmsf.xvg")[:, 1]
#print(x)
#print(y)
rmsf_cutoff=4.1
sticky_resid=[]
#With ,1 it gives 79 pockets
#for i in range(0,len(y),1):
for i in range(0,len(y),2):
    if (y[i]<rmsf_cutoff):
        sticky_resid.append(str(int(x[i])))
        print(str(int(x[i])),5)
x=np.loadtxt("secondary/helix.dat")[:, 0]
y=np.loadtxt("secondary/helix.dat")[:, 1]

transient_ahelix_resid=[]
helix_upper=0.5
helix_lower=0.01
for i in range(0,len(y)):
    if (y[i]<helix_upper and y[i]>helix_lower):
        transient_ahelix_resid.append(str(x[i]))
        #print(x[i])

In [ ]:
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import glob
import toml


path = './results/*.toml'
files = glob.glob(path)


in_cavity = {}
pdbs = []

for res in files:
    
    pdb = res.split("/")[-1].split("_")[1:2]
    #print(res,res.split("/")[-1],pdb)
    results = toml.load(res)
    for cavity in results['RESULTS']['AVG_HYDROPATHY']:
        try:
            #Here change hydropathy to smaller cuttoff?
            if results['RESULTS']['AVG_HYDROPATHY'][cavity] < -0.2:
                if results['RESULTS']['VOLUME'][cavity] > 120:
                    pdbs.append(results['FILES']['INPUT'])
                    for residue in results['RESULTS']['RESIDUES'][cavity]:
                        resid = residue[0] + "_" + residue[2]
                        try:
                            in_cavity[resid]['in_cavities'] += 1
                            in_cavity[resid]['cavity']['cavity_name'].append(cavity)
                            in_cavity[resid]['cavity']['cavity_struct'].append(pdb)
                        except KeyError as e:
                            in_cavity[resid] = {}
                            in_cavity[resid]['in_cavities'] = 1
                            in_cavity[resid]['cavity'] = {}
                            in_cavity[resid]['cavity']['cavity_name'] = [cavity]
                            in_cavity[resid]['cavity']['cavity_struct'] = [pdb]
        except Exception as e:
            pass


print("---PDBS---")
print(pdbs)
print("---RES---")
residues = []
in_cavs = []
for key in in_cavity:
    if in_cavity[key]['in_cavities'] > 0:
        residues.append(key.split("_")[0])
        in_cavs.append(in_cavity[key]['in_cavities'])


fig = plt.figure(figsize=(10, 5))

# creating the bar plot
plt.bar(residues, in_cavs, color='maroon',
        width=0.8)
plt.xticks(fontsize=5, rotation=90)
plt.xlabel("residues")
plt.ylabel("in cavities")
plt.title("Resids in cavities")
plt.savefig('cavity_resid.pdf')
plt.show()


most=[]
most=sticky_resid
#most = ['83', '85', '87', '88', '91', '94', '95', '89', '92', '93']

most_prominent = {}


file_cavs = []

for res in files:
    pdb = res.split("/")[-1].split("_")[1:2]
    print(pdb)
    results = toml.load(res)
    for k in results['RESULTS']['RESIDUES']:

        res_pos = []
        for res in results['RESULTS']['RESIDUES'][k]:
            res_pos.append(res[0])
        all = len(res_pos)
        trues = []
        for re in res_pos:
            if re in most:
                trues.append(True)
        if len(trues) > 5:
            try:
                if results['RESULTS']['AVG_HYDROPATHY'][k] < 0 \
                        and 80 < results['RESULTS']['AREA'][k] < 2480:
                    print(results['RESULTS']['VOLUME'][k])
                    file_cav = {results['FILES']['INPUT']: k}
                    file_cavs.append(file_cav)
            except Exception as e:
                pass

print(len(file_cavs))
print(file_cavs)
for i in range(0,len(file_cavs)):
    print(file_cavs[i])

with open("./cavs_found_more.json", "w") as f:
    f.write(json.dumps(file_cavs))
    f.close()

In [ ]:
import mdtraj as md
import sys
import json
import toml
# Opening JSON file
f = open('cavs_found_more.json')
# returns JSON object as
# a dictionary
data = json.load(f)
f.close()
# Iterating through the json
# list
f = open('com.dat','w')
for i in data:
    try:
        resultList = list(i.items())
        name = resultList[0][0].split("/")[-1].split(".")[0]
        #print(name,resultList[0][1])
        traj=md.load('cavities/cavity_'+str(name)+'.pdb', top='cavities/cavity_'+str(name)+'.pdb')
        cm=md.compute_center_of_mass(traj=traj, select='resname '+str(resultList[0][1]))
        X_cm=cm[0][0]
        Y_cm=cm[0][1]
        Z_cm=cm[0][2]
        results = toml.load('results/results_'+str(name)+'.toml')
        ep_s=[]
        ep_e=[]
        ep_c=[]
        for cavity in results['RESULTS']['RESIDUES']:
            if(cavity == str(resultList[0][1])):
                print(cavity,name)
                lista=results['RESULTS']['RESIDUES'][str(resultList[0][1])]
                resids=[int(item[0]) for item in lista]
                print(resids) 
                cut=2
                
                old=resids[0]
                count=1
                for j in range(1,len(resids)):
                    if (resids[j] - resids[j-1] <= cut):
                        count+=1
                        #print('hello',count)
                    else:
                        ep_s.append(old)
                        ep_e.append(resids[j-1])
                        ep_c.append(count)
                        #print(old,resids[j-1],count)
                        old=resids[j]
                        count=1
                #print('end for',old,resids[j-1],count)
                ep_s.append(old)
                ep_e.append(resids[j-1])
                ep_c.append(count)
                max_ind=ep_c.index(max(ep_c))
                print(ep_s[max_ind], ep_e[max_ind])

        f.write(str(name)+" "+str(resultList[0][1])+" "+str(X_cm)+" "+str(Y_cm)+" "+str(Z_cm)+" "+str(ep_s[max_ind])+" "+str(ep_e[max_ind])+"\n")
    except Exception as e:
        pass

f.close()
